# lgb - optuna 데모 
* 번거로운 모델 튜닝 과정을 효율적으로 학습할 수 있도록 유도
* 선택 후 검증 결과를 가지고 이후 값 선택에 도움을 주는데 최적화되어 있다.  
* 모델 튜닝 라이브러리 방식 2가지  
  티피이_하이퍼&옵티(가우시안보다 빨라도 파라미터 증가에 따라 시간 급증. 예로 100번 돌림 오래 걸림)

## 라이브러리 import 및 설정

In [9]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from lightgbm import LGBMClassifier
from matplotlib import pyplot as plt
from matplotlib import rcParams
import numpy as np
import optuna.integration.lightgbm as lgb
from pathlib import Path
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold, train_test_split
import seaborn as sns
import warnings
rcParams['figure.figsize'] = (16, 8)
plt.style.use('fivethirtyeight')
pd.set_option('max_columns', 100)
pd.set_option("display.precision", 4)
warnings.simplefilter('ignore')

## 학습데이터 로드

In [2]:
data_dir = Path('C:\\Users\\USER\\Desktop\\오픈소스SW설계\\단국대\\DataInput\\')
sub_dir = Path('C:\\Users\\USER\\Desktop\\오픈소스SW설계\\단국대\\sub\\')
feature_dir = Path('C:\\Users\\USER\\Desktop\\오픈소스SW설계\\단국대\\DataInput\\')

val_dir = Path('C:\\Users\\USER\\Desktop\\오픈소스SW설계\\단국대\\val')
tst_dir = Path('C:\\Users\\USER\\Desktop\\오픈소스SW설계\\단국대\\tst')

trn_file = data_dir / 'train.csv'
tst_file = data_dir / 'test.csv'
sample_file = data_dir / 'sample_submission.csv'

target_col = 'class'
n_fold = 5
n_class = 3
seed = 42

algo_name = 'lgb_optuna'
feature_name = 'feature'
model_name = f'{algo_name}_{feature_name}'

feature_file = feature_dir / f'{feature_name}.csv'
p_val_file = val_dir / f'{model_name}.val.csv'
p_tst_file = tst_dir / f'{model_name}.tst.csv'
sub_file = sub_dir / f'{model_name}.csv'
df = pd.read_csv(feature_file, index_col=0)

In [3]:
y = df[target_col].values[:320000]           #종속
df.drop(target_col, axis=1, inplace=True)    #독립을 위한 종속 제거
trn = df.iloc[:320000].values                #train을 위한 독립 320000개     
tst = df.iloc[320000:].values                #test을 위한 독립 80000개
feature_name = df.columns.tolist()           #독립변수 이름 리스트로 변환
print(y.shape, trn.shape, tst.shape)

(320000,) (320000, 19) (80000, 19)


## 학습/검증 데이터 구분
**학습 후 모델의 예측/분류 정확도 계산을 위한 검증데이터 분리**

[Hold-out Validation]

In [6]:
X_trn, X_val, y_trn, y_val = train_test_split(trn, y, test_size=.2, random_state=seed)

## 모델(파라미터) 튜닝_Optuna

In [7]:
params = {
    "objective": "multiclass",
    "metric": "multi_logloss",
    "num_class": 3,
    "n_estimators": 1000,
    "subsample_freq": 1,
    "lambda_l1": 0.,
    "lambda_l2": 0.,
    "random_state": seed,
    "n_jobs": -1,
}
# 고정 파라미터

dtrain = lgb.Dataset(X_trn, label=y_trn)
dval = lgb.Dataset(X_val, label=y_val)
# 학습과 검증 데이터 셋

model = lgb.train(params, dtrain, valid_sets=[dtrain, dval], 
                  verbose_eval=100, early_stopping_rounds=10)
# model_lgbm
prediction = np.argmax(model.predict(X_val, num_iteration=model.best_iteration), 
                       axis=1)
accuracy = accuracy_score(y_val, prediction)

params = model.params
print("Best params:", params)
print("  Accuracy = {}".format(accuracy))
print("  Params: ")
for key, value in params.items():
    print("    {}: {}".format(key, value))

[I 2020-10-18 01:53:49,864] A new study created in memory with name: no-name-c8668de6-31ed-4704-a2bc-31702ec0aeb5

feature_fraction, val_score: inf:   0%|                                                          | 0/7 [00:00<?, ?it/s]

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.036827 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4586
[LightGBM] [Info] Number of data points in the train set: 256000, number of used features: 19
[LightGBM] [Info] Start training from score -0.980965
[LightGBM] [Info] Start training from score -2.012659
[LightGBM] [Info] Start training from score -0.710460
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[84]	training's multi_logloss: 0.157951	valid_1's multi_logloss: 0.171065



feature_fraction, val_score: 0.171065:  14%|######4                                      | 1/7 [00:11<01:11, 11.87s/it][I 2020-10-18 01:54:01,744] Trial 0 finished with value: 0.1710654637466055 and parameters: {'feature_fraction': 1.0}. Best is trial 0 with value: 0.1710654637466055.

feature_fraction, val_score: 0.171065:  14%|######4                                      | 1/7 [00:11<01:11, 11.87s/it]

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.086499 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4586
[LightGBM] [Info] Number of data points in the train set: 256000, number of used features: 19
[LightGBM] [Info] Start training from score -0.980965
[LightGBM] [Info] Start training from score -2.012659
[LightGBM] [Info] Start training from score -0.710460
Training until validation scores don't improve for 10 rounds
[100]	training's multi_logloss: 0.154537	valid_1's multi_logloss: 0.168848
Early stopping, best iteration is:
[100]	training's multi_logloss: 0.154537	valid_1's multi_logloss: 0.168848



feature_fraction, val_score: 0.168848:  29%|############8                                | 2/7 [00:37<01:19, 15.93s/it][I 2020-10-18 01:54:27,156] Trial 1 finished with value: 0.16884814405994292 and parameters: {'feature_fraction': 0.8999999999999999}. Best is trial 1 with value: 0.16884814405994292.

feature_fraction, val_score: 0.168848:  29%|############8                                | 2/7 [00:37<01:19, 15.93s/it]

...

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.075045 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4586
[LightGBM] [Info] Number of data points in the train set: 256000, number of used features: 19
[LightGBM] [Info] Start training from score -0.980965
[LightGBM] [Info] Start training from score -2.012659
[LightGBM] [Info] Start training from score -0.710460
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 10 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further sp


min_data_in_leaf, val_score: 0.161949:  80%|####################################         | 4/5 [01:55<00:29, 29.04s/it][I 2020-10-18 02:43:53,661] Trial 66 finished with value: 0.16256674525402054 and parameters: {'min_child_samples': 25}. Best is trial 64 with value: 0.16235787622797557.

min_data_in_leaf, val_score: 0.161949:  80%|####################################         | 4/5 [01:55<00:29, 29.04s/it]

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.047062 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4586
[LightGBM] [Info] Number of data points in the train set: 256000, number of used features: 19
[LightGBM] [Info] Start training from score -0.980965
[LightGBM] [Info] Start training from score -2.012659
[LightGBM] [Info] Start training from score -0.710460
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 10 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further sp


min_data_in_leaf, val_score: 0.161949: 100%|#############################################| 5/5 [02:26<00:00, 29.30s/it]


Best params: {'objective': 'multiclass', 'metric': 'multi_logloss', 'num_class': 3, 'lambda_l1': 1.3133712832020334e-05, 'lambda_l2': 5.009547955531789, 'random_state': 42, 'n_jobs': -1, 'feature_pre_filter': False, 'bagging_freq': 1, 'num_leaves': 255, 'feature_fraction': 0.5, 'bagging_fraction': 1.0, 'min_child_samples': 20, 'num_iterations': 1000, 'early_stopping_round': 10}
  Accuracy = 0.932046875
  Params: 
    objective: multiclass
    metric: multi_logloss
    num_class: 3
    lambda_l1: 1.3133712832020334e-05
    lambda_l2: 5.009547955531789
    random_state: 42
    n_jobs: -1
    feature_pre_filter: False
    bagging_freq: 1
    num_leaves: 255
    feature_fraction: 0.5
    bagging_fraction: 1.0
    min_child_samples: 20
    num_iterations: 1000
    early_stopping_round: 10


## Stratified K-Fold Cross Validation
*Stratified N-Fold CV: N-Fold CV에서 각각의 폴드에서 종속변수의 분포가 동일하도록 폴드를 나누는 방식.  
현재 사용하는 데이터처럼 분류학습에서 종속변수의 범주의 분포가 균일하지 않을 때 사용된다.

In [8]:
cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

In [10]:
p_val = np.zeros((trn.shape[0], n_class))
p_tst = np.zeros((tst.shape[0], n_class))
#(cv.split(),1)
# -  StratifiedKFold 경우, trn,y 독립과 종속(동일 분포를 위해) 인자
for i, (i_trn, i_val) in enumerate(cv.split(trn, y), 1):
    print(f'training model for CV #{i}')
    clf = LGBMClassifier(**params)
    clf.fit(trn[i_trn], y[i_trn],
            eval_set=[(trn[i_val], y[i_val])],
            eval_metric='multiclass',
            early_stopping_rounds=10)
    
    p_val[i_val, :] = clf.predict_proba(trn[i_val])
    #확률로 예측값 초가화
    p_tst += clf.predict_proba(tst) / n_fold
    #평균 효과를 위해 먼저 폴드 개수만큼 나누어 초기화

training model for CV #1
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] lambda_l1 is set=1.3133712832020334e-05, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.3133712832020334e-05
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] lambda_l2 is set=5.009547955531789, reg_lambda=0.0 will be ignored. Current value: lambda_l2=5.009547955531789
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[1]	valid_0's multi_logloss: 0.856194
Training until validation scores don't improve for 10 rounds
[2]	valid_0's multi_logloss: 0.753805
[3]	valid_0's multi_logloss: 0.670156
[4]	valid_0's multi_logloss: 0.597971
[5]	valid_0's multi_logloss: 0.540904
[6]	valid_0's multi_logloss: 0.500171
[7]	valid_0's multi_logloss: 0.46463
[8]	valid_0's multi_logloss

[54]	valid_0's multi_logloss: 0.170692
[55]	valid_0's multi_logloss: 0.170348
[56]	valid_0's multi_logloss: 0.169985
[57]	valid_0's multi_logloss: 0.169751
[58]	valid_0's multi_logloss: 0.169415
[59]	valid_0's multi_logloss: 0.169045
[60]	valid_0's multi_logloss: 0.168709
[61]	valid_0's multi_logloss: 0.168158
[62]	valid_0's multi_logloss: 0.167959
[63]	valid_0's multi_logloss: 0.167553
[64]	valid_0's multi_logloss: 0.167135
[65]	valid_0's multi_logloss: 0.166871
[66]	valid_0's multi_logloss: 0.166557
[67]	valid_0's multi_logloss: 0.166188
[68]	valid_0's multi_logloss: 0.165901
[69]	valid_0's multi_logloss: 0.165605
[70]	valid_0's multi_logloss: 0.165344
[71]	valid_0's multi_logloss: 0.165181
[72]	valid_0's multi_logloss: 0.16497
[73]	valid_0's multi_logloss: 0.164886
[74]	valid_0's multi_logloss: 0.164612
[75]	valid_0's multi_logloss: 0.164465
[76]	valid_0's multi_logloss: 0.164242
[77]	valid_0's multi_logloss: 0.164131
[78]	valid_0's multi_logloss: 0.164006
[79]	valid_0's multi_loglo

In [11]:
print(f'{accuracy_score(y, np.argmax(p_val, axis=1)) * 100:.4f}%')# 93.2837%

93.2837%


[stacking 사용을 위해 p_val,p_tst 저장]

In [12]:
np.savetxt(p_val_file, p_val, fmt='%.6f', delimiter=',')
np.savetxt(p_tst_file, p_tst, fmt='%.6f', delimiter=',')

## 제출 파일 생성

In [14]:
sub = pd.read_csv(sample_file, index_col=0)
print(sub.shape)
sub[target_col] = np.argmax(p_tst, axis=1)
sub.head()

(80000, 1)


,class
id,
320000,2
320001,0
320002,2
320003,0
320004,2


In [15]:
sub.to_csv(sub_file)